 kernel  k-means

In [31]:
import numpy as np
from PIL import Image


def load_image(image_path):

    img = np.array(Image.open(image_path).convert('RGB'))
    h, w, _ = img.shape
    rgb = []
    spatial = []
    for i in range(h):
        for j in range(w):
            r, g, b = img[i, j] # RGB
            rgb.append([r, g, b])
            spatial.append([i, j])

    return np.array(rgb), np.array(spatial)


rgb_feature, spatial_feature = load_image('image1.png')
print('rgb_feature shape:', rgb_feature.shape)
print('spatial_feature shape:', spatial_feature.shape)

rgb_feature shape: (10000, 3)
spatial_feature shape: (10000, 2)


In [ ]:
def compute_kernel_matrix(rgb, spatial, gamma_c, gamma_s):
    N = len(rgb)
    K = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            ds = np.linalg.norm(rgb[i] - rgb[j])
            dc = np.linalg.norm(spatial[i] - spatial[j])
            K[i, j] = np.exp(-gamma_s * ds) * np.exp(-gamma_c * dc)
    return K


def kernel_kmeans( K, num_clusters, max_iter=10):
    N = K.shape[0]
    labels = np.random.randint(0, num_clusters, N)
    for it in range(max_iter):
        new_labels = np.zeros(N, dtype=int)
        for i in range(N):
            for c in range(num_clusters):
                min_dist = float('inf')
                idx = np.where(labels == c)[0]
                if len(idx) == 0:
                    continue
                term1 = K[i,i]
                term2 = -2 * np.sum(K[i, idx]) / len(idx)
                term3 = np.sum(K[np.ix_(idx, idx)]) / (len(idx) ** 2)
                dist = term1 + term2 + term3
                if dist < min_dist:
                    min_dist = dist
                    new_labels[i] = c

        if np.array_equal(new_labels, labels):
            print(f"Converged at iteration {it + 1}")  #no change in labels
            break
        labels = new_labels
    return labels


In [ ]:
import matplotlib.pyplot as plt

gamma_s = 0.1
gamma_c = 0.001
K = compute_kernel_matrix(spatial_feature, rgb_feature, gamma_s, gamma_c)

TypeError: kernel_kmeans() got an unexpected keyword argument 'n_clusters'

In [ ]:
labels = kernel_kmeans(K, num_clusters=2, max_iter=10)
label_img = labels.reshape(100, 100)
plt.imshow(label_img, cmap='tab10')
plt.title("Kernel K-Means (Fixed Gamma)")
plt.axis('off')
plt.show()